<a href="https://colab.research.google.com/github/DrumDong/Donggyu/blob/master/concrete_%EB%8B%A4%EC%A4%91%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Dense

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/IT신기술/concrete.csv')
dataset.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [55]:
dataset.info() # null값이 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
cement          1030 non-null float64
slag            1030 non-null float64
ash             1030 non-null float64
water           1030 non-null float64
superplastic    1030 non-null float64
coarseagg       1030 non-null float64
fineagg         1030 non-null float64
age             1030 non-null int64
strength        1030 non-null float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [56]:
dataset.shape

(1030, 9)

# data Nomalization

- def norm(x):

    _max = x.max()

    _min = x.min()

    _denom = _max - _min

    return (x - _min) / _denom

In [57]:
from sklearn.preprocessing import MinMaxScaler
print(dataset.head())

   cement   slag  ash  water  superplastic  coarseagg  fineagg  age  strength
0   540.0    0.0  0.0  162.0           2.5     1040.0    676.0   28     79.99
1   540.0    0.0  0.0  162.0           2.5     1055.0    676.0   28     61.89
2   332.5  142.5  0.0  228.0           0.0      932.0    594.0  270     40.27
3   332.5  142.5  0.0  228.0           0.0      932.0    594.0  365     41.05
4   198.6  132.4  0.0  192.0           0.0      978.4    825.5  360     44.30


In [58]:
min_max_scaler = MinMaxScaler()
fitted = min_max_scaler.fit(dataset)
print(fitted.data_max_) # 각 컬럼에서 제일 큰 값들이 나옴

[ 540.   359.4  200.1  247.    32.2 1145.   992.6  365.    82.6]


In [59]:
dataset_mms = min_max_scaler.transform(dataset)
dataset_mms # numpy array로 반환됨

array([[1.        , 0.        , 0.        , ..., 0.20572002, 0.07417582,
        0.96748474],
       [1.        , 0.        , 0.        , ..., 0.20572002, 0.07417582,
        0.74199576],
       [0.52625571, 0.39649416, 0.        , ..., 0.        , 0.73901099,
        0.47265479],
       ...,
       [0.10616438, 0.38786867, 0.54272864, ..., 0.46663322, 0.07417582,
        0.26622649],
       [0.1303653 , 0.51947691, 0.        , ..., 0.48896136, 0.07417582,
        0.37922013],
       [0.36278539, 0.27963272, 0.39130435, ..., 0.42022077, 0.07417582,
        0.37461069]])

### dataframe form으로 바꾸기

In [60]:
dataset_mms = pd.DataFrame(dataset_mms,
                           columns= dataset.columns,
                           index = list(dataset.index.values))

print(dataset_mms.head())

     cement      slag  ash     water  ...  coarseagg   fineagg       age  strength
0  1.000000  0.000000  0.0  0.321086  ...   0.694767  0.205720  0.074176  0.967485
1  1.000000  0.000000  0.0  0.321086  ...   0.738372  0.205720  0.074176  0.741996
2  0.526256  0.396494  0.0  0.848243  ...   0.380814  0.000000  0.739011  0.472655
3  0.526256  0.396494  0.0  0.848243  ...   0.380814  0.000000  1.000000  0.482372
4  0.220548  0.368392  0.0  0.560703  ...   0.515698  0.580783  0.986264  0.522860

[5 rows x 9 columns]


#dataframe form numpy 배열로 변환

In [61]:
# change dataset to numpy.array
dataset_numpy = dataset_mms.values
dataset_numpy

array([[1.        , 0.        , 0.        , ..., 0.20572002, 0.07417582,
        0.96748474],
       [1.        , 0.        , 0.        , ..., 0.20572002, 0.07417582,
        0.74199576],
       [0.52625571, 0.39649416, 0.        , ..., 0.        , 0.73901099,
        0.47265479],
       ...,
       [0.10616438, 0.38786867, 0.54272864, ..., 0.46663322, 0.07417582,
        0.26622649],
       [0.1303653 , 0.51947691, 0.        , ..., 0.48896136, 0.07417582,
        0.37922013],
       [0.36278539, 0.27963272, 0.39130435, ..., 0.42022077, 0.07417582,
        0.37461069]])

# 2. data split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
# train, test set 분할
X_train,X_test,y_train,y_test = train_test_split(
    dataset_numpy[:,:-1],dataset_numpy[:,[-1]],random_state=0
)

In [64]:
print('X_train 크기:',X_train.shape)
print('y_train 크기:',y_train.shape)
print('X_test 크기:',X_test.shape)
print('y_test 크기:',y_test.shape)

X_train 크기: (772, 8)
y_train 크기: (772, 1)
X_test 크기: (258, 8)
y_test 크기: (258, 1)


# tensoflow
- 다중회귀분석

In [0]:
X = tf.placeholder('float',[None,8])
Y = tf.placeholder('float',[None,1])

In [0]:
W = tf.Variable(tf.random_normal([8,1]),name='weight')
b = tf.Variable(tf.random_normal([1]),name='bias')

In [0]:
hypothesis = tf.matmul(X,W)+b

## Cost/Loss function

In [0]:
cost = tf.reduce_mean(tf.square(hypothesis -Y))
train = tf.train.GradientDescentOptimizer(learning_rate=0.25).minimize(cost)

## Accuracy computation

In [109]:
with tf.Session() as sess:
  # Initialize Tensorflow variavles
   sess.run(tf.global_variables_initializer())

   for step in range(10001):
     cost_val,_ = sess.run([cost,train],
                           feed_dict={X:X_train,Y:y_train})
     if step!=0 and step % 10000 ==0:
       print('최종 step:',step,'최종 train error cost:',cost_val)

   print('------------------------')

   cost_val,hy_val = sess.run([cost,hypothesis],feed_dict={X:X_test,Y:y_test})
   print('test error cost:',cost_val)#,'hypothesis:',hy_val)
   
   #File = pd.DataFrame(hypothesis,y_test,columns=['hypothesis','y_test'])


최종 step: 10000 최종 train error cost: 0.01728079
------------------------
test error cost: 0.014885706


train_cost에서 최종적으로 0.0172로 머무른다. 0으로 가깝게 하고싶은데 계속 거기서 머물러 있는 이유를 찾아야할 것 같다. local minimum에 빠진 것인지, 아니면 활성화 함수를 활용해야하나..?

정규화를 할 때 split을 먼저하고 train과 test 데이터를 따로따로 해야하는지.. 같이해도 되는지..흠냐 고민이구만